In [9]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
import random


In [10]:
# Define the paths to the folders containing the EEG data
folders = {
    'complex_math': r'G:\University\Project_Intermship\An EEG Recordings Dataset for Mental Stress Detection\An EEG Recordings Dataset for Mental Stress Detection\clean data\CSVs\Complex Mathematical Problem solving (CMPS)',
    'horror': r'G:\University\Project_Intermship\An EEG Recordings Dataset for Mental Stress Detection\An EEG Recordings Dataset for Mental Stress Detection\clean data\CSVs\Horrer Video Stimulation',
    'mental_test': r'G:\University\Project_Intermship\An EEG Recordings Dataset for Mental Stress Detection\An EEG Recordings Dataset for Mental Stress Detection\clean data\CSVs\Trier Mental Challenge Test (TMCT)'
}

# Initialize lists to store the data and labels
data = []
labels = []

# Find the maximum sequence length
max_length = 0
for folder in folders.values():
    for filename in os.listdir(folder):
        if filename.endswith('.csv'):
            filepath = os.path.join(folder, filename)
            df = pd.read_csv(filepath)
            max_length = max(max_length, len(df))

# Loop through each folder and read the CSV files
for label, folder in folders.items():
    for filename in os.listdir(folder):
        if filename.endswith('.csv'):
            filepath = os.path.join(folder, filename)
            df = pd.read_csv(filepath)
            # Convert DataFrame to a NumPy array
            array = df.values
            # Pad the array to the maximum length with zeros
            if len(array) < max_length:
                padded_array = np.pad(array, ((0, max_length - len(array)), (0, 0)), mode='constant')
            else:
                padded_array = array
            data.append(padded_array.flatten())
            labels.append(label)

# Convert the data and labels into NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Encode the labels as integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)


In [11]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.3, random_state=42)

# Initialize and train a Random Forest classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(report)


Accuracy: 0.76
Classification Report:
              precision    recall  f1-score   support

complex_math       1.00      0.29      0.44         7
      horror       1.00      1.00      1.00         6
 mental_test       0.62      1.00      0.76         8

    accuracy                           0.76        21
   macro avg       0.87      0.76      0.74        21
weighted avg       0.85      0.76      0.72        21



In [12]:
def test_model_random_combination(folders, classifier, label_encoder, max_length, num_samples=1):
    # Initialize lists to store the real and predicted labels
    real_labels = []
    predicted_labels = []
    
    # Initialize dictionaries to track incorrect predictions for each condition
    incorrect_counts = {label: 0 for label in folders.keys()}
    total_counts = {label: 0 for label in folders.keys()}

    # Loop over each condition
    for label, folder in folders.items():
        filenames = [f for f in os.listdir(folder) if f.endswith('.csv')]
        
        # Randomly select the specified number of CSV files
        selected_filenames = random.sample(filenames, num_samples)
        
        for random_filename in selected_filenames:
            filepath = os.path.join(folder, random_filename)
            df = pd.read_csv(filepath)
            array = df.values

            # Ensure the data has the same length as the training data
            if len(array) < max_length:
                padded_array = np.pad(array, ((0, max_length - len(array)), (0, 0)), mode='constant')
            else:
                padded_array = array[:max_length]

            # Flatten the data to match the model's expected input
            test_data = padded_array.flatten().reshape(1, -1)

            # Predict the condition using the trained model
            predicted_label_encoded = classifier.predict(test_data)
            predicted_label = label_encoder.inverse_transform(predicted_label_encoded)

            # Store the real and predicted labels
            real_labels.append(label)
            predicted_labels.append(predicted_label[0])

            # Update counts for the current condition
            total_counts[label] += 1
            if label != predicted_label[0]:
                incorrect_counts[label] += 1

    # Calculate the overall incorrect rate
    incorrect_predictions = sum(incorrect_counts.values())
    incorrect_rate = incorrect_predictions / len(real_labels)

    # Print the real and predicted labels
    print(f'Real Labels: {real_labels}')
    print(f'Predicted Labels: {predicted_labels}')
    print(f'Overall Incorrect Rate: {incorrect_rate:.2f}')
    print(f'Total Test Samples: {len(real_labels)}')

    # Print incorrect rates for each condition
    for label in folders.keys():
        condition_incorrect_rate = incorrect_counts[label] / total_counts[label] if total_counts[label] > 0 else 0
        print(f'Incorrect Rate for {label}: {condition_incorrect_rate:.2f}')

# Example usage: test with 3 samples from each condition
test_model_random_combination(folders, classifier, label_encoder, max_length, num_samples=22)


Real Labels: ['complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'complex_math', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'horror', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test', 'mental_test']
Predicted Labels: ['mental_test', 'complex_math', 'complex_math', 'complex_math', 'c